In [1]:
import gym
import numpy as np
import pandas as pd
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from StockTrade import StockTrade
from gym import spaces

In [2]:
env = StockTrade(stock_quantity=1, change_stocks=False)

In [3]:
env.reset()

array([3.246e+01, 3.338e+01, 3.246e+01, 3.325e+01, 1.138e+05])

In [4]:
obs, rewards, dones, info = env.step([-1])
rewards

-20.0

In [ ]:
# The noise objects for DDPG
n_actions = env.action_space.shape
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = DDPG('MlpPolicy', env, action_noise=action_noise, verbose=1)
model.learn(total_timesteps=1000000, log_interval=10)
model.save("123Automator")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model = DDPG.load("123Automator")
model.learn(total_timesteps=1000000, log_interval=1000)
model.save("123Automator")
env = model.get_env()

In [ ]:
obs = env.reset()
dones = False
while not dones:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()
env.close()